In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('novel_story_sep.csv', lineterminator='\n')

In [3]:
df.columns

Index(['itemId', 'title', 'story', 'review', 'piece'], dtype='object')

# LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습시키기

In [4]:
import gensim
gensim.__version__

'3.8.1'

In [5]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet
import pyLDAvis
import pyLDAvis.gensim

In [6]:
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
# pyLDAvis.display(vis)
# pyLDAvis.enable_notebook()

In [7]:
df.drop_duplicates('story', inplace=True)

In [8]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [9]:
stories = df['piece']
# df['story'] + df['review'] + 

In [10]:
# df

In [11]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [12]:
id2word = corpora.Dictionary(data_word)

In [13]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=4, random_seed=12, id2word=id2word)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print(coherence_ldamallet) #응집성 지수

0.3385571328156868


In [14]:
# Select the model and print the topics
# optimal_model = model_list[model_list_num]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics()[0])

(0, '0.018*"그녀" + 0.018*"눈" + 0.015*"얼굴" + 0.015*"손" + 0.013*"소리" + 0.008*"몸" + 0.007*"머리" + 0.007*"목소리" + 0.006*"입" + 0.006*"고개"')


In [15]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=20)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data['title'], Data['story'], sent_topics_df], axis=1)

    return(sent_topics_df)

In [16]:
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()


# def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values



# # Can take a long time to run.
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=21, step=2)




# limit=21; start=4; step=2;
# x = range(start, limit, step)
# topic_num = 0
# count = 0
# max_coherence = 0
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", cv)
#     coherence = cv
#     if coherence >= max_coherence:
#         max_coherence = coherence
#         topic_num = m
#         model_list_num = count   
#     count = count+1

        
# # Select the model and print the topics
# optimal_model = model_list[model_list_num]
# model_topics = optimal_model.show_topics(formatted=False)
# #print(optimal_model.print_topics(num_words=10))


In [32]:
optimal_model[corpus]

[[(0, 0.25612535612535614),
  (1, 0.25612535612535614),
  (2, 0.37065527065527065),
  (3, 0.1170940170940171)],
 [(0, 0.2162534435261708),
  (1, 0.2695133149678604),
  (2, 0.37052341597796146),
  (3, 0.14370982552800735)],
 [(0, 0.2763653483992467),
  (1, 0.15552416823603266),
  (2, 0.2631826741996234),
  (3, 0.3049278091650973)],
 [(0, 0.17098765432098764),
  (1, 0.4104938271604938),
  (2, 0.15493827160493828),
  (3, 0.26358024691358023)],
 [(0, 0.06759006963366636),
  (1, 0.04942476536481986),
  (2, 0.045337571904329396),
  (3, 0.8376475930971844)],
 [(0, 0.28222761596548),
  (1, 0.28762135922330095),
  (2, 0.23799892125134844),
  (3, 0.19215210355987056)],
 [(0, 0.09317710824167065),
  (1, 0.06655507731547904),
  (2, 0.06209150326797386),
  (3, 0.7781763111748764)],
 [(0, 0.14647909712287394),
  (1, 0.09910983945318709),
  (2, 0.041726275631855034),
  (3, 0.7126847877920839)],
 [(0, 0.2091060985797828),
  (1, 0.46842105263157896),
  (2, 0.1558061821219716),
  (3, 0.16666666666666666

In [18]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df)

<class 'pandas.core.frame.DataFrame'>


In [19]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [20]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [21]:
sent_topics_sorteddf_mallet[['Perc_Contribution', 'Dominant_Topic', 'Topic_Keywords']]

,Perc_Contribution,Dominant_Topic,Topic_Keywords
327,0.7879,0.0,"그녀, 눈, 얼굴, 손, 소리, 몸, 머리, 목소리, 입, 고개, 순간, 좀, 눈물..."
341,0.5739,1.0,"당신, 인간, 사실, 여자, 존재, 삶, 세계, 죽음, 남자, 순간, 의미, 인생,..."
143,0.5373,2.0,"마음, 엄마, 사랑, 세상, 살, 나를, 날, 집, 아이, 건, 면, 아빠, 아버지..."
4,0.8376,3.0,"시간, 요, 여기, 몇, 시작, 꿈, 줄, 끝, 돈, 정말, 아주, 테, 누구, 조..."


In [22]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0.0    171
1.0    192
2.0    200
3.0     82
dtype: int64

In [23]:
# topic_df = df_topic_story[df_topic_story['Dominant_Topic']==12]
# topic_df.sort_values('Perc_Contribution', ascending=False)

In [24]:
# df_topic_sort[df_topic_sort['Perc_Contribution'] > 0.5]